In [52]:
import pandas as pd
import numpy as np
import os as os
from IPython.display import display
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB,CategoricalNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,f1_score

### Load data

In [53]:
dfs_train = {}
dfs_test = {}

In [171]:
for root, directory, files in os.walk("data/train_preprocessed", topdown=False):
    if files:
        for file in files:
            dfs_train[file[:-4]] = pd.read_csv(f"{root}/{file}")
            try:
                dfs_train[file[:-4]] = pd.get_dummies(dfs_train[file[:-4]], columns=["Avg_bookie_prediction"], prefix='Bookie_Prediction')
            except KeyError:
                pass
lens_test = 0
for root, directory, files in os.walk("data/test_preprocessed", topdown=False):
    if files:
        for file in files:
            dfs_test[file[:-4]] = pd.read_csv(f"{root}/{file}")
    try:
        dfs_test[file[:-4]] = pd.get_dummies(dfs_test[file[:-4]], columns=["Avg_bookie_prediction"], prefix='Bookie_Prediction')
    except KeyError:
        pass
            # lens_test += dfs_test[file[:-4]].shape[0]
print("---")
lens_orig = 0
df_test_y = pd.DataFrame()
for root, directory, files in os.walk("data/orig_data", topdown=False):
    if files:
        for file in files:
            # print(pd.read_csv(f"{root}/{file}").shape)
            tmp = pd.read_csv(f"{root}/{file}")
            lens_orig += tmp.shape[0]
            tmp["country"] = file[:-5]
            
            df_test_y = pd.concat([df_test_y, tmp], axis=0)

---


0       True
1      False
2      False
3       True
4       True
       ...  
337    False
338    False
339    False
340    False
341    False
Name: Bookie_Prediction_A, Length: 342, dtype: bool

In [55]:
df_test_y["Date"] = pd.to_datetime(df_test_y["Date"], format = '%d/%m/%Y', dayfirst=True)

df_test_y.sort_values(by="Date", inplace=True, ascending=True)

In [178]:
df_test_y[df_test_y["country"]=="england"]["league"]

KeyError: 'league'

In [57]:
for country in df_test_y["country"].unique():
    col1 = df_test_y[df_test_y["country"] == country]["FTHG"].reset_index()
    col2 = df_test_y[df_test_y["country"] == country]["FTAG"].reset_index()

    target_values = col1["FTHG"] + col2["FTAG"]

    dfs_test[f"df_{country}"]["Target_regr"] = target_values
    dfs_test[f"df_{country}"]["FTHG"] = col1["FTHG"]
    dfs_test[f"df_{country}"]["FTAG"] = col2["FTAG"]
    dfs_test[f"df_{country}"]['Target_clas'] = [0 if a > h else 1 if h > a else -1 for a, h in zip(dfs_test[f"df_{country}"]['FTAG'], dfs_test[f"df_{country}"]['FTHG'])]
    dfs_test[f"df_{country}"].drop(columns=["FTHG", "FTAG", "Unnamed: 0"], inplace=True)


In [177]:
dfs_train["df_england"]

KeyError: 'div'

In [59]:
def impute_nan_values(dfs):
    for df in dfs.values():
        for col in df.columns:
            if df[col].dtype == np.float64 or df[col].dtype == np.int64:
                df[col] = df.groupby("season")[col].transform(lambda x: x.fillna(x.mean()))
        df.dropna(inplace=True)
impute_nan_values(dfs_train)
impute_nan_values(dfs_test)

In [60]:
# validation set
dfs_valid_reg_X = {}
dfs_valid_reg_y = {}
dfs_train_reg_X = {}
dfs_train_reg_y = {}
dfs_valid_clas_X = {}
dfs_valid_clas_y = {}
dfs_train_clas_X = {}
dfs_train_clas_y = {}
dfs_test_clas_X = {}
dfs_test_clas_y = {}
dfs_test_reg_X = {}
dfs_test_reg_y = {}

cols_to_drop = ['FTHG', 'FTAG', 'MatchTeams', 'SameHomeTeam', 'Target', 'Target_regr', 'Target_clas', "Unnamed: 0", "index", "season"]

for country in dfs_train:
    dfs_train[country] =   dfs_train[country][dfs_train[country]["season"] > 17]
    dfs_train[country]['Target_clas'] = [0 if a > h else 1 if h > a else -1 for a, h in zip(dfs_train[country]['FTAG'], dfs_train[country]['FTHG'])]

    dfs_valid_reg_y[country[3:]] = dfs_train[country][dfs_train[country]["season"] == 21]["Target_regr"]
    dfs_valid_reg_X[country[3:]] = dfs_train[country][dfs_train[country]["season"] == 21].drop(columns=cols_to_drop, errors='ignore')
    
    dfs_train_reg_y[country[3:]] = dfs_train[country][dfs_train[country]["season"] != 21]["Target_regr"]
    dfs_train_reg_X[country[3:]] = dfs_train[country][dfs_train[country]["season"] != 21].drop(columns=cols_to_drop, errors='ignore')
    
    dfs_train_clas_y[country[3:]] = dfs_train[country][dfs_train[country]["season"] != 21]["Target_clas"]
    dfs_train_clas_X[country[3:]] = dfs_train[country][dfs_train[country]["season"] != 21].drop(columns=cols_to_drop, errors='ignore')

    dfs_valid_clas_y[country[3:]] = dfs_train[country][dfs_train[country]["season"] == 21]["Target_clas"]
    dfs_valid_clas_X[country[3:]] = dfs_train[country][dfs_train[country]["season"] == 21].drop(columns=cols_to_drop, errors='ignore')

for country in dfs_test:
    dfs_test_reg_y[country[3:]] = dfs_test[country][dfs_test[country]["season"] != 21]["Target_regr"]
    dfs_test_reg_X[country[3:]] = dfs_test[country][dfs_test[country]["season"] != 21].drop(columns=cols_to_drop, errors='ignore')

    dfs_test_clas_y[country[3:]] = dfs_test[country][dfs_test[country]["season"] != 21]["Target_clas"]
    dfs_test_clas_X[country[3:]] = dfs_test[country][dfs_test[country]["season"] != 21].drop(columns=cols_to_drop, errors='ignore')

C:\Users\vojta\AppData\Local\Temp\ipykernel_27868\27335406.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_train[country]['Target_clas'] = [0 if a > h else 1 if h > a else -1 for a, h in zip(dfs_train[country]['FTAG'], dfs_train[country]['FTHG'])]


In [61]:
train_results = pd.DataFrame(columns=["country"])
test_results = pd.DataFrame(columns=["country"])

In [62]:
train_results["country"] = dfs_train_clas_X.keys()
test_results["country"] = dfs_train_clas_X.keys()

In [63]:
train_results

,country
0,belgium
1,england
2,france
3,germany
4,greece
5,italy
6,netherlands
7,portugal
8,scotland
9,spain


# Classification task
We decided to go with Voting Classifier consisting of 3 classification algorithms -  Gaussian Naive Bayes, RandomForestClassifier & Logistic Regression

### Logistic Regression

In [64]:
dfs_test_predict = {}
dfs_train_predict = {}
dfs_valid_predict = {}
dfs_best_params_lr = {}
C_values = [0.001, 0.01, 0.1, 1, 10, 100]  # Example values for regularization parameter C
penalty_values = ['l1', 'l2']  # Example values for penalty (regularization term)

for country in dfs_train_clas_X.keys():
    max_value = 0
    result = [[0 for _ in range(len(penalty_values))] for _ in range(len(C_values))]
    max_i = -1
    max_j = -1

    for i, C in enumerate(C_values):
        for j, penalty in enumerate(penalty_values):
            lr = LogisticRegression(C=C, penalty=penalty, solver='liblinear', max_iter=10000)
            lr.fit(dfs_train_clas_X[country], dfs_train_clas_y[country])
            dfs_valid_predict[country] = lr.predict(dfs_valid_clas_X[country])
            result[i][j] = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')

    for i in range(len(C_values)):
        for j in range(len(penalty_values)):
            current_value = result[i][j]
            if current_value > max_value:
                max_value = current_value
                max_i = i
                max_j = j

    best_C = C_values[max_i]
    best_penalty = penalty_values[max_j]
    dfs_best_params_lr[country] = {"C": best_C, "penalty": best_penalty}
    test_results.loc[train_results["country"] == country, "lr"] = max_value
    print(f"Best performance on validation data for {country} is {max_value} for C = {best_C} and penalty = {best_penalty}")


Best performance on validation data for belgium is 0.4463821716453295 for C = 1 and penalty = l2
Best performance on validation data for england is 0.4054037836997059 for C = 10 and penalty = l2
Best performance on validation data for france is 0.4075106206252051 for C = 1 and penalty = l1
Best performance on validation data for germany is 0.41897415919793746 for C = 100 and penalty = l2
Best performance on validation data for greece is 0.43259113403698635 for C = 0.1 and penalty = l2
Best performance on validation data for italy is 0.43277442981349384 for C = 1 and penalty = l2
Best performance on validation data for netherlands is 0.4136953877901773 for C = 1 and penalty = l1
Best performance on validation data for portugal is 0.45984829350235934 for C = 10 and penalty = l1
Best performance on validation data for scotland is 0.4195134685220309 for C = 1 and penalty = l2
Best performance on validation data for spain is 0.42241395767095186 for C = 1 and penalty = l1
Best performance on

### Random Forest Model

Utilization of basic grid search to find best parameters

In [65]:
rfm = RandomForestClassifier(random_state=42)

In [66]:
train_results_random_forest = {}
def random_forest(params):
    rfm.set_params(**params)
    rfm.fit(dfs_train_clas_X[country], dfs_train_clas_y[country])
    dfs_train_predict[country] = rfm.predict(dfs_train_clas_X[country])
    dfs_valid_predict[country] = rfm.predict(dfs_valid_clas_X[country])
    dfs_test_predict[country] = rfm.predict(dfs_test_clas_X[country])
        
    # print(f"Average Depth of Decision Trees for {country}: {average_depth}")
    train_results[params["max_depth"]] = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')
    result_train = f1_score(dfs_train_clas_y[country], dfs_train_predict[country], average='macro')
    result_valid = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')
    result_test =  f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')
    return result_valid

In [67]:
dfs_train_predict = {}
dfs_valid_predict = {}
dfs_best_params_rfm = {}
max_depth_values = [i for i in range(10, 25)]
min_samples_split_values = [2, 5, 15, 10, 20]

for country in dfs_train_clas_X.keys():
    max_value = 0
    result = [[0 for _ in range(len(min_samples_split_values))] for _ in range(len(max_depth_values))]
    max_i = -1
    max_j = -1

    for i, max_depth in enumerate(max_depth_values):
        for j, min_samples_split in enumerate(min_samples_split_values):
            result[i][j] = random_forest({"max_depth": max_depth + 1, "min_samples_split": min_samples_split})

    for i in range(len(max_depth_values)):
        for j in range(len(min_samples_split_values)):
            current_value = result[i][j]
            if current_value > max_value:
                max_value = current_value
                max_i = i
                max_j = j

    best_max_depth = max_depth_values[max_i]
    best_min_samples_split = min_samples_split_values[max_j]
    test_results.loc[test_results["country"] == country, "rfm"] = max_value
    dfs_best_params_rfm[country] = {"max_depth": best_max_depth, "min_samples_split": best_min_samples_split}

    print(f"Best performance on validation data for {country} is {max_value} for max_depth = {best_max_depth} and min_samples_split = {best_min_samples_split}")

Best performance on validation data for belgium is 0.45683323496622735 for max_depth = 23 and min_samples_split = 2
Best performance on validation data for england is 0.389815488249824 for max_depth = 21 and min_samples_split = 2
Best performance on validation data for france is 0.42556743721259843 for max_depth = 17 and min_samples_split = 15
Best performance on validation data for germany is 0.39388206484610616 for max_depth = 23 and min_samples_split = 2
Best performance on validation data for greece is 0.4526915050241622 for max_depth = 16 and min_samples_split = 5
Best performance on validation data for italy is 0.44078503048354717 for max_depth = 15 and min_samples_split = 2
Best performance on validation data for netherlands is 0.47335343509641453 for max_depth = 10 and min_samples_split = 2
Best performance on validation data for portugal is 0.4809563576017399 for max_depth = 19 and min_samples_split = 5
Best performance on validation data for scotland is 0.41141796875435377 fo

### MultinomialNaive Bayes

In [68]:
nb = MultinomialNB()

In [69]:
dfs_train_predict = {}
dfs_valid_predict = {}
for country in dfs_train_clas_X.keys():
    nb.fit(dfs_train_clas_X[country], dfs_train_clas_y[country])
    dfs_train_predict[country] = nb.predict(dfs_train_clas_X[country])
    dfs_valid_predict[country] = nb.predict(dfs_valid_clas_X[country])
    dfs_test_predict[country] = nb.predict(dfs_test_clas_X[country])

    test_results.loc[test_results["country"] == country, "nb"] = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')
    # test_results.loc[test_results["country"] == country, "nb"] = f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')

    print(f"F1-score for baseline Multinomial Naive Bayes model on train data for {country}: {f1_score(dfs_train_clas_y[country], dfs_train_predict[country], average='macro')}")
    print(f"F1-score for baseline Multinomial Naive Bayes model on validation data for {country}: {f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')}")
    print(f"F1-score for baseline Multinomial Naive Bayes model on test data for {country}: {f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')}\n --------")


F1-score for baseline Multinomial Naive Bayes model on train data for belgium: 0.4255711292491478
F1-score for baseline Multinomial Naive Bayes model on validation data for belgium: 0.40595500070221185
F1-score for baseline Multinomial Naive Bayes model on test data for belgium: 0.3616061357996842
 --------
F1-score for baseline Multinomial Naive Bayes model on train data for england: 0.47621455325225986
F1-score for baseline Multinomial Naive Bayes model on validation data for england: 0.44302221161866573
F1-score for baseline Multinomial Naive Bayes model on test data for england: 0.3425640810954196
 --------
F1-score for baseline Multinomial Naive Bayes model on train data for france: 0.48709385058962384
F1-score for baseline Multinomial Naive Bayes model on validation data for france: 0.44166874510691706
F1-score for baseline Multinomial Naive Bayes model on test data for france: 0.38679821408619536
 --------
F1-score for baseline Multinomial Naive Bayes model on train data for ger

### KNeighborsClassifier

In [70]:
kn = KNeighborsClassifier()

In [71]:
dfs_train_predict = {}
dfs_valid_predict = {}
for country in dfs_train_clas_X.keys():
    kn.fit(dfs_train_clas_X[country], dfs_train_clas_y[country])
    dfs_train_predict[country] = kn.predict(dfs_train_clas_X[country])
    dfs_valid_predict[country] = kn.predict(dfs_valid_clas_X[country])
    dfs_test_predict[country] = kn.predict(dfs_test_clas_X[country])
    test_results.loc[test_results["country"] == country, "kn"] = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')
    # test_results.loc[test_results["country"] == country, "kn"] = f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')

    print(f"F1-score for baseline KNeighbors model on train data for {country}: {f1_score(dfs_train_clas_y[country], dfs_train_predict[country], average='macro')}")
    print(f"F1-score for baseline KNeighbors model on validation data for {country}: {f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')}")
    print(f"F1-score for baseline KNeighbors model on test data for {country}: {f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')}\n --------")

F1-score for baseline KNeighbors model on train data for belgium: 0.5710600107944851
F1-score for baseline KNeighbors model on validation data for belgium: 0.46310744352754857
F1-score for baseline KNeighbors model on test data for belgium: 0.3701209330737993
 --------
F1-score for baseline KNeighbors model on train data for england: 0.5827749869996115
F1-score for baseline KNeighbors model on validation data for england: 0.3933019717107375
F1-score for baseline KNeighbors model on test data for england: 0.31704960062302684
 --------
F1-score for baseline KNeighbors model on train data for france: 0.6045559302159741
F1-score for baseline KNeighbors model on validation data for france: 0.4070633684926858
F1-score for baseline KNeighbors model on test data for france: 0.3567566253218599
 --------
F1-score for baseline KNeighbors model on train data for germany: 0.5979028682142488
F1-score for baseline KNeighbors model on validation data for germany: 0.43189945723011075
F1-score for basel

### Gradient Boosting Classifier

In [72]:
gbc = GradientBoostingClassifier(random_state=42)

In [73]:
dfs_train_predict = {}
dfs_valid_predict = {}
for country in dfs_train_clas_X.keys():
    gbc.fit(dfs_train_clas_X[country], dfs_train_clas_y[country])
    dfs_train_predict[country] = gbc.predict(dfs_train_clas_X[country])
    dfs_valid_predict[country] = gbc.predict(dfs_valid_clas_X[country])
    dfs_test_predict[country] = gbc.predict(dfs_test_clas_X[country])

    test_results.loc[test_results["country"] == country, "gbc"] = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')
    # test_results.loc[test_results["country"] == country, "gbc"] = f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')
    print(f"F1-score for baseline GradientBoostingClassifier model on train data for {country}: {f1_score(dfs_train_clas_y[country], dfs_train_predict[country], average='macro')}")
    print(f"F1-score for baseline GradientBoostingClassifier model on validation data for {country}: {f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')}")
    print(f"F1-score for baseline GradientBoostingClassifier model on test data for {country}: {f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')}\n --------")

F1-score for baseline GradientBoostingClassifier model on train data for belgium: 0.8681219669740553
F1-score for baseline GradientBoostingClassifier model on validation data for belgium: 0.42784629583018713
F1-score for baseline GradientBoostingClassifier model on test data for belgium: 0.34212471619409685
 --------
F1-score for baseline GradientBoostingClassifier model on train data for england: 0.5295075625406082
F1-score for baseline GradientBoostingClassifier model on validation data for england: 0.39290681987422227
F1-score for baseline GradientBoostingClassifier model on test data for england: 0.3035912952227761
 --------
F1-score for baseline GradientBoostingClassifier model on train data for france: 0.7090153120670831
F1-score for baseline GradientBoostingClassifier model on validation data for france: 0.37029676863346017
F1-score for baseline GradientBoostingClassifier model on test data for france: 0.3393761903526264
 --------
F1-score for baseline GradientBoostingClassifier

### Ada Boosting Classifier

In [74]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score

dfs_train_predict = {}
dfs_valid_predict = {}
dfs_test_predict = {}

for country in dfs_train_clas_X.keys():
    # Create an AdaBoostClassifier
    abc = AdaBoostClassifier(n_estimators=50, random_state=42)  # You can adjust the hyperparameters as needed

    # Train the AdaBoostClassifier
    abc.fit(dfs_train_clas_X[country], dfs_train_clas_y[country])

    # Make predictions on train, validation, and test sets
    dfs_train_predict[country] = abc.predict(dfs_train_clas_X[country])
    dfs_valid_predict[country] = abc.predict(dfs_valid_clas_X[country])
    dfs_test_predict[country] = abc.predict(dfs_test_clas_X[country])

    # Calculate and store F1-score on validation set
    test_results.loc[test_results["country"] == country, "abc"] = f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')
    # test_results.loc[test_results["country"] == country, "abc"] = f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')

    # Print F1-scores
    print(f"F1-score for baseline AdaBoostClassifier model on train data for {country}: {f1_score(dfs_train_clas_y[country], dfs_train_predict[country], average='macro')}")
    print(f"F1-score for baseline AdaBoostClassifier model on validation data for {country}: {f1_score(dfs_valid_clas_y[country], dfs_valid_predict[country], average='macro')}")
    print(f"F1-score for baseline AdaBoostClassifier model on test data for {country}: {f1_score(dfs_test_clas_y[country], dfs_test_predict[country], average='macro')}\n --------")


F1-score for baseline AdaBoostClassifier model on train data for belgium: 0.5693628535338963
F1-score for baseline AdaBoostClassifier model on validation data for belgium: 0.41463340555206835
F1-score for baseline AdaBoostClassifier model on test data for belgium: 0.34059320474975524
 --------
F1-score for baseline AdaBoostClassifier model on train data for england: 0.4230871646336721
F1-score for baseline AdaBoostClassifier model on validation data for england: 0.36791561566605385
F1-score for baseline AdaBoostClassifier model on test data for england: 0.3077086469035622
 --------
F1-score for baseline AdaBoostClassifier model on train data for france: 0.510443312034176
F1-score for baseline AdaBoostClassifier model on validation data for france: 0.3852251491921093
F1-score for baseline AdaBoostClassifier model on test data for france: 0.33881003634190376
 --------
F1-score for baseline AdaBoostClassifier model on train data for germany: 0.4975428032560056
F1-score for baseline AdaBoo

### Implementing Voting Classifier
Our voting classifier is designed to use the 3 best performing classifiers and use them for the final prediction 

In [104]:
best_models = []
for i, row in test_results.iterrows():
    best_models.append(pd.to_numeric(row[["lr", "rfm", "nb", "kn", "gbc", "abc"]]).nlargest(4).index.tolist())

In [172]:
def voting_classifier(best_models_country, X_train, y_train, X_val, y_val, country: str, X_test, y_test):
    """"""
    models = {
        "nb": nb,
        "kn": kn,
        "gbc": GradientBoostingClassifier(random_state=42),
        "rfm": RandomForestClassifier(random_state=42),
        "lr": LogisticRegression(random_state=42, solver='liblinear', max_iter=10000),
        "abc": AdaBoostClassifier()
        
    }
    print(best_models_country)
    clf1 = models[best_models_country[0]] if best_models_country[0] != "rfm" else models[best_models_country[0]].set_params(**dfs_best_params_rfm[country]) if best_models_country[0] != "lr" else models[best_models_country[0]].set_params(**dfs_best_params_lr[country])
    clf2 = models[best_models_country[1]] if best_models_country[1] != "rfm" else models[best_models_country[1]].set_params(**dfs_best_params_rfm[country]) if best_models_country[0] != "lr" else models[best_models_country[0]].set_params(**dfs_best_params_lr[country])
    clf3 = models[best_models_country[2]] if best_models_country[2] != "rfm" else models[best_models_country[2]].set_params(**dfs_best_params_rfm[country]) if best_models_country[0] != "lr" else models[best_models_country[0]].set_params(**dfs_best_params_lr[country])
    
    eclf = VotingClassifier(
        estimators=[(best_models_country[0], clf1 ), (best_models_country[1], clf2 ),(best_models_country[2], clf3 )],
        voting='soft', weights=[1.1,1,1]
    ) 
    eclf.fit(X_train,y_train)
    
    y_train_predict = eclf.predict(X_train)
    y_val_predict = eclf.predict(X_val)
    
    y_test_predict = eclf.predict(X_test)
    clf1.fit(X_train, y_train)
    y_valid_best_model = clf1.predict(X_val)
    y_test_best_model = clf1.predict(X_test)
    
    print(f"F1-score for voting classifier model on train data for {country}: {f1_score(y_train, y_train_predict, average='macro')}")
    print(f"F1-score for voting classifier model on validation data for {country}: {f1_score(y_val, y_val_predict, average='macro')}")
    print(f"F1-score for voting classifier model on test data for {country}: {f1_score(y_test, y_test_predict, average='macro')}")
    print("\n")
    print(f"F1-score for best model ({best_models_country[0]}) on valid data for {country}: {f1_score(y_val, y_valid_best_model, average='macro')}")
    print(f"F1-score for best model ({best_models_country[0]}) on test data for {country}: {f1_score(y_test, y_test_best_model, average='macro')}\n --------")
    X_test["final_prediction"] =  y_test_predict if f1_score(y_val, y_val_predict, average='macro') > f1_score(y_val, y_valid_best_model, average='macro') else y_test_best_model
    return X_test
    

In [173]:
final_predict = {}
for i, country in enumerate(dfs_train_clas_X.keys()):
    final_predict[country] = voting_classifier(best_models[i], dfs_train_clas_X[country], dfs_train_clas_y[country], dfs_valid_clas_X[country], dfs_valid_clas_y[country],country, dfs_test_clas_X[country], dfs_test_clas_y[country])

['kn', 'rfm', 'lr', 'gbc']
F1-score for voting classifier model on train data for belgium: 0.8428295379519897
F1-score for voting classifier model on validation data for belgium: 0.45599202441307707
F1-score for voting classifier model on test data for belgium: 0.348206875678466


F1-score for best model (kn) on valid data for belgium: 0.46310744352754857
F1-score for best model (kn) on test data for belgium: 0.3701209330737993
 --------
['nb', 'lr', 'kn', 'gbc']
F1-score for voting classifier model on train data for england: 0.5385513871974877
F1-score for voting classifier model on validation data for england: 0.4229649632551446
F1-score for voting classifier model on test data for england: 0.325085105091345


F1-score for best model (nb) on valid data for england: 0.44302221161866573
F1-score for best model (nb) on test data for england: 0.3425640810954196
 --------
['nb', 'rfm', 'lr', 'kn']
F1-score for voting classifier model on train data for france: 0.5904066582374531
F1-score f

In [144]:
for country in final_predict:
    final_predict[country] = np.where(final_predict[country] == 0, 'A', np.where(final_predict[country] == 1, 'H', 'D'))
    df = pd.DataFrame(final_predict[country])
    df.to_csv(f"{country}.csv", index= False)

In [174]:
final_predict["england"][final_predict["england"][""]]

,Avg_away_odds,Avg_home_odds,Avg_draw_odds,Var_away_odds,Var_home_odds,Var_draw_odds,LastMatchAwayGoals,LastMatchHomeGoals,LastMatchAwayWin,LastMatchHomeWin,...,AwayLossRatio,AwayDrawRatio,HomeTeamAvgShotsOnTarget,AwayTeamAvgShotsOnTarget,HomeTeamScoredRatio,AwayTeamScoredRatio,Bookie_Prediction_A,Bookie_Prediction_D,Bookie_Prediction_H,final_prediction
0,2.505000,2.908333,3.201667,0.000150,0.007777,0.007217,2.0,1.0,1,0,...,0.362887,0.271134,4.722680,4.689437,0.484754,0.478541,True,False,False,0
1,2.928333,2.413333,3.210000,0.015217,0.002187,0.003600,1.0,1.0,0,0,...,0.385193,0.271459,4.822256,4.246407,0.474050,0.462791,False,False,True,-1
2,3.271667,2.228333,3.356667,0.007417,0.001017,0.004267,1.0,2.0,0,1,...,0.328947,0.289474,4.931718,5.233534,0.504348,0.490965,False,False,True,0
3,2.990000,2.456667,3.203333,0.004000,0.003067,0.004067,2.0,1.0,1,0,...,0.365510,0.291757,5.389130,5.093117,0.508261,0.515563,False,False,True,1
4,2.601667,2.631667,3.435000,0.002457,0.000617,0.008150,3.0,1.0,1,0,...,0.384536,0.276289,4.785567,5.205083,0.492595,0.503591,True,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2031,2.343333,2.836667,3.523333,0.011307,0.003787,0.012867,0.0,3.0,0,0,...,0.390507,0.251348,6.823096,5.361601,0.668308,0.487054,True,False,False,0
2032,1.836667,3.806667,3.900000,0.002547,0.031467,0.020000,0.0,1.0,0,0,...,0.324769,0.263104,4.954779,6.555630,0.532182,0.635985,True,False,False,0
2033,3.350000,2.000000,3.746667,0.038000,0.001000,0.025667,2.0,0.0,0,0,...,0.267073,0.193902,4.956103,4.484721,0.478221,0.512730,False,False,True,1
2034,7.693333,1.361667,5.101667,0.135267,0.001977,0.121617,0.0,1.0,0,0,...,0.337838,0.275184,6.768564,5.013760,0.639847,0.516239,False,False,True,1
